# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ing. en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**: TikTok Tracks Analysis

**Fecha**: 21 de febrero del 2025

**Nombre del Estudiante**: Marco Albanese, Vicente Siloe

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TikTok Tracks Analysis") \
    .master("spark://cd68d43f7ac6:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 13:42:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import importlib
import equipo_mcqueen.spark_utils

columns_info = [ ("track_id", "StringType"),
                ("track_name", "StringType"),
                ("artist_id", "StringType"),
                ("artist_name", "StringType"),
                ("album_id", "StringType"),
                ("duration", "IntegerType"),
                ("release_date", "TimestampType"),
                ("popularity", "IntegerType"),
                ("danceability", "DoubleType"),
                ("energy", "DoubleType"),
                ("key", "IntegerType"),
                ("loudness", "DoubleType"),
                ("mode", "IntegerType"),
                ("speechiness", "DoubleType"),
                ("acousticness", "DoubleType"),
                ("instrumentalness", "DoubleType"),
                ("liveness", "DoubleType"),
                ("valence", "DoubleType"),
                ("tempo", "DoubleType"),
                ("playlist_id", "StringType"),
                ("playlist_name", "StringType"),
                ("duration_mins", "DoubleType"),
                ("genre", "StringType")]

schema = equipo_mcqueen.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

#### **Filter and Count Popular Tracks:** Filter songs with a popularity score greater than 80 and count the number of such tracks

In [ ]:
tiktok_df.filter(tiktok_df.popularity > 80).count()

1023

#### **Calculate Average Duration of Songs by Genre:** Group songs by genre and calculate the average duration mins for each genre

In [7]:
grouped_df = tiktok_df.groupBy("genre").avg("duration_mins")
grouped_df.show()

+------------------+------------------+
|             genre|avg(duration_mins)|
+------------------+------------------+
|TIKTOK PHILIPPINES|3.2801328435737513|
|      TIKTOK DANCE| 3.015020713916861|
|           _TIKTOK| 3.251196442168827|
|        TIKTOK OPM| 4.257192861885788|
+------------------+------------------+



#### **Find the Top 5 Most Energetic Songs:** Sort songs by energy in descending order and retrieve the top 5 songs.

In [ ]:
sorted_energy_df = tiktok_df.sort(tiktok_df["energy"].desc())
sorted_energy_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------------------+-------------------+--------+------------------+------------------+--------------------+--------------------+-----------------+----------+
|            track_id|          track_name|           artist_id|         artist_name|            album_id|duration|       release_date|popularity|      danceability|            energy|key|          loudness|mode|speechiness|        acousticness|   instrumentalness|liveness|           valence|             tempo|         playlist_id|       playlist_name|    duration_mins|     genre|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------

#### **Calculate the Total Duration of Songs in Each Playlist:** Group songs by playlist name and calculate the total duration mins for each playlist.

In [9]:
# Group songs by playlit name and calculate the total duration in mins for each playlist
grouped_playlist_df = tiktok_df.groupBy("playlist_name").sum("duration_mins")
grouped_playlist_df.show()

+--------------------+------------------+
|       playlist_name|sum(duration_mins)|
+--------------------+------------------+
|5IZc3KIVFhjzJ0L2k...| 7.474666666666667|
|08ia51KbTcfs4QVT5...|            4.1485|
|7xVLFuuYdAvcTfcP3...| 9.456433333333333|
|2RBILNmyq8p4fqVWO...| 2.162933333333333|
|6GdDjthxbTGBV9rl2...|3.3209166666666667|
|7krYEnB1OI1RbnJBa...|2.0957666666666666|
|1FgPyHX7HruKDL4Tx...|            2.4448|
|62RtxFf9epYNWOUHJ...|2.6694333333333335|
|5ow0sNF1zSqp71Ix5...|2.7334833333333335|
|0LlJbV4lyzJYE14YC...|10.709133333333334|
|6NFKf8vBApSvtzkap...|3.7074333333333334|
|5P8lyudWE7HQxb4lu...| 4.250666666666667|
|2BgEsaKNfHUdlh97K...| 3.116433333333333|
|7F9vK8hNFMml4GtHs...| 3.173783333333333|
|4vVTI94F9uJ8lHNDW...|3.3657666666666666|
|2uULRpRtKhCdojXwo...|               2.2|
|1tRlGMHsf21FDo6pj...|           1.79005|
|215fAfwkWtlj30ofd...|2.3214166666666665|
|3bidbhpOYeV4knp8A...|            4.3057|
|0YFocHKmrMme7Isel...| 4.810383333333333|
+--------------------+------------

In [ ]:
# Stop the SparkContext
sc.stop()